# How to use PyTorch LSTMs/GRUs for time series regression
We aim to forcast stock price value by using LSTM with Pytorch.

credit: https://github.com/CrosstabKite/lstm-forecasting/blob/master/lstm_forecasting.ipynb


- modified on (12/02/2023)
- modified on (27/01/2025) : change to lightning pytorch


# Data

In [ ]:
!pip install torchinfo --quiet
!pip install lightning --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 30.3 MB/s eta 0:00:00


In [ ]:
!wget https://github.com/pvateekul/2110531_DSDE_2023s1/raw/main/code/Week05_Intro_Deep_Learning/data/GOOG.csv

--2025-01-27 07:59:10--  https://github.com/pvateekul/2110531_DSDE_2023s1/raw/main/code/Week05_Intro_Deep_Learning/data/GOOG.csv
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pvateekul/2110531_DSDE_2023s1/main/code/Week05_Intro_Deep_Learning/data/GOOG.csv [following]
--2025-01-27 07:59:10--  https://raw.githubusercontent.com/pvateekul/2110531_DSDE_2023s1/main/code/Week05_Intro_Deep_Learning/data/GOOG.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 284248 (278K) [text/plain]
Saving to: ‘GOOG.csv’

GOOG.csv            100%[===================>] 277.59K  --.-KB/s    in 0.006s  

2025-01-2

In [ ]:
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchinfo import summary
import pandas as pd

L.seed_everything(101)
torch.manual_seed(101)

INFO: Seed set to 101
INFO:lightning.fabric.utilities.seed:Seed set to 101


In [ ]:
df = pd.read_csv('GOOG.csv', index_col="Date")
df = df.drop(['Adj Close'], axis = 1)
df

,Open,High,Low,Close,Volume
Date,,,,,
2004-08-19,49.813286,51.835709,47.800831,49.982655,44871300
2004-08-20,50.316402,54.336334,50.062355,53.952770,22942800
2004-08-23,55.168217,56.528118,54.321388,54.495735,18342800
2004-08-24,55.412300,55.591629,51.591621,52.239193,15319700
2004-08-25,52.284027,53.798351,51.746044,52.802086,9232100
...,...,...,...,...,...
2019-09-30,1220.969971,1226.000000,1212.300049,1219.000000,1404100
2019-10-01,1219.000000,1231.229980,1203.579956,1205.099976,1273500
2019-10-02,1196.979980,1196.979980,1171.290039,1176.630005,1615100


In [ ]:
df.loc['2019-02-26']

,2019-02-26
Open,1.105750e+03
High,1.119510e+03
Low,1.099920e+03
Close,1.115130e+03
Volume,1.471300e+06


In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"

plot_template = dict(
    layout=go.Layout({
        "font_size": 18,
        "xaxis_title_font_size": 24,
        "yaxis_title_font_size": 24})
)

fig = px.line(df['Open'], labels=dict(
    created_at="Date", value="Open", variable="Sensor"
))
fig.update_layout(
  template=plot_template, legend=dict(orientation='h', y=1.02, title_text="")
)
fig.show()

## Create the target variable

In [ ]:
target_col = "Open"
features = list(df.columns.difference([target_col]))

forecast_lead = 1
target = f"{target_col}_lead{forecast_lead}"

df[target] = df[target_col].shift(-forecast_lead)
df = df.iloc[:-forecast_lead]

In [ ]:
df

,Open,High,Low,Close,Volume,Open_lead1
Date,,,,,,
2004-08-19,49.813286,51.835709,47.800831,49.982655,44871300,50.316402
2004-08-20,50.316402,54.336334,50.062355,53.952770,22942800,55.168217
2004-08-23,55.168217,56.528118,54.321388,54.495735,18342800,55.412300
2004-08-24,55.412300,55.591629,51.591621,52.239193,15319700,52.284027
2004-08-25,52.284027,53.798351,51.746044,52.802086,9232100,52.279045
...,...,...,...,...,...,...
2019-09-27,1243.010010,1244.020020,1214.449951,1225.089966,1353900,1220.969971
2019-09-30,1220.969971,1226.000000,1212.300049,1219.000000,1404100,1219.000000
2019-10-01,1219.000000,1231.229980,1203.579956,1205.099976,1273500,1196.979980


## Create a hold-out test set and preprocess the data

In [ ]:
test_start = "2019-01-01"
val_start = "2018-01-01"

df_train = df.loc[:val_start].copy()
df_val = df.loc[val_start:test_start].copy()
df_test = df.loc[test_start:].copy()

print("Test set fraction:", len(df_test) / len(df))

Test set fraction: 0.050157563025210086


## Standardize the features and target, based on the training set

In [ ]:
target_mean = df_train[target].mean()
target_stdev = df_train[target].std()

for c in df_train.columns:
    mean = df_train[c].mean()
    stdev = df_train[c].std()

    df_train[c] = (df_train[c] - mean) / stdev
    df_val[c] = (df_val[c] - mean) / stdev
    df_test[c] = (df_test[c] - mean) / stdev

## Create datasets that PyTorch `DataLoader` can work with

In [ ]:
class SequenceDataset(Dataset):
    def __init__(self, dataframe, target, features, sequence_length=5):
        self.features = features
        self.target = target
        self.sequence_length = sequence_length
        self.y = torch.tensor(dataframe[self.target].values).float()
        self.X = torch.tensor(dataframe[self.features].values).float()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i):
        if i >= self.sequence_length - 1:
            i_start = i - self.sequence_length + 1
            x = self.X[i_start:(i + 1), :]
        else:
            padding = self.X[0].repeat(self.sequence_length - i - 1, 1)
            x = self.X[0:(i + 1), :]
            x = torch.cat((padding, x), 0)

        return x, self.y[i]

- showing in nut shell of SequenceDataset, etc.,...


In [ ]:
i = 27
sequence_length = 4

train_dataset = SequenceDataset(
    df_train,
    target=target,
    features=features,
    sequence_length=sequence_length
)

X, y = train_dataset[i]
print(X)

tensor([[-1.4005, -1.4044, -1.4018,  0.0794],
        [-1.4026, -1.4013, -1.3959,  0.1501],
        [-1.4059, -1.4081, -1.4001, -0.0974],
        [-1.3878, -1.3944, -1.3950,  1.0892]])


In [ ]:
X, y = train_dataset[i + 1]
print(X)

tensor([[-1.4026, -1.4013, -1.3959,  0.1501],
        [-1.4059, -1.4081, -1.4001, -0.0974],
        [-1.3878, -1.3944, -1.3950,  1.0892],
        [-1.3789, -1.3784, -1.3822,  2.7240]])


In [ ]:
print(df_train[features].iloc[(i - sequence_length + 1): (i + 1)])

               Close      High       Low    Volume
Date                                              
2004-09-23 -1.400509 -1.404409 -1.401750  0.079391
2004-09-24 -1.402597 -1.401323 -1.395935  0.150113
2004-09-27 -1.405909 -1.408082 -1.400095 -0.097414
2004-09-28 -1.387767 -1.394397 -1.394980  1.089247


In [ ]:
torch.manual_seed(99)
train_loader = DataLoader(train_dataset, batch_size=3, shuffle=True)

X, y = next(iter(train_loader))
print(X.shape)
print(X)

torch.Size([3, 4, 4])
tensor([[[-1.0283e+00, -1.0363e+00, -1.0355e+00,  3.3244e+00],
         [-1.0139e+00, -1.0227e+00, -1.0264e+00,  3.3321e+00],
         [-1.0183e+00, -1.0127e+00, -1.0091e+00,  3.6286e+00],
         [-1.0379e+00, -1.0229e+00, -1.0300e+00,  3.4512e+00]],

        [[-3.0608e-01, -3.1554e-01, -3.1614e-01,  2.0066e+00],
         [-2.9534e-01, -2.7960e-01, -2.8488e-01,  2.8479e+00],
         [-2.8260e-01, -2.8693e-01, -2.9963e-01,  6.5448e-01],
         [-2.2982e-01, -2.3949e-01, -2.4929e-01,  6.8556e-01]],

        [[-4.5368e-03, -1.7926e-03,  1.3788e-03, -4.5791e-01],
         [-3.9661e-02, -1.2372e-02, -3.4364e-02, -1.4888e-01],
         [ 3.1979e-02,  2.4656e-02, -2.3751e-02,  4.4746e-01],
         [ 3.2486e-02,  2.5747e-02, -5.2009e-03, -2.5444e-01]]])


## Create the datasets and data loaders for real

In this tutorial we will
use sequences of length 60 (60 days) to forcast 1 day ahead.

The PyTorch `DataLoader` is a very convenient way to iterate through these datasets. For
the training set we'll shuffle (the rows *within* each training sequence are not
shuffled, only the order in which we draw those blocks). For the test set, shuffling
isn't necessary.

In [ ]:
L.seed_everything(101)
class TimeSeriesDataModule(L.LightningDataModule):
    def __init__(self, df_train, df_val, df_test, target, features,
                 batch_size=32, sequence_length=60):
        super().__init__()
        self.df_train = df_train
        self.df_val = df_val
        self.df_test = df_test
        self.target = target
        self.features = features
        self.batch_size = batch_size
        self.sequence_length = sequence_length

    def setup(self, stage: str):
        self.train_dataset = SequenceDataset(self.df_train, self.target, self.features, self.sequence_length)
        self.val_dataset = SequenceDataset(self.df_val, self.target, self.features, self.sequence_length)
        self.test_dataset = SequenceDataset(self.df_test, self.target, self.features, self.sequence_length)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False)

    def predict_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False)

data_module = TimeSeriesDataModule(df_train, df_val, df_test, target, features, batch_size=32, sequence_length=60)

## Explore (Optional)
data_module.setup(stage='fit') # setup train + validate set in data module
X, y = next(iter(data_module.train_dataloader()))
print("Features shape:", X.shape)
print("Target shape:", y.shape)


INFO: Seed set to 101
INFO:lightning.fabric.utilities.seed:Seed set to 101


Features shape: torch.Size([32, 60, 4])
Target shape: torch.Size([32])


# LSTM

## The model and learning algorithm

![picture](https://i.stack.imgur.com/SjnTl.png)

Credit : https://stackoverflow.com/questions/48302810/whats-the-difference-between-hidden-and-output-in-pytorch-lstm

In [ ]:
class ShallowRegressionLSTM(L.LightningModule):
    def __init__(self, num_features, hidden_units, num_layers, learning_rate=5e-4):
        super().__init__()
        self.num_features = num_features  # this is the number of features
        self.hidden_units = hidden_units
        self.num_layers = num_layers
        self.learning_rate = learning_rate
        self.validation_step_outputs = []

        self.lstm = nn.LSTM(
            input_size=num_features,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=self.num_layers
        )

        self.linear = nn.Linear(in_features=self.hidden_units, out_features=1)
        self.loss_function = nn.MSELoss()

    def forward(self, x):
        batch_size = x.shape[0]

        # initialize the hidden and cell state of the LSTM layer
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).to(x.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).to(x.device)
        _, (hn, _) = self.lstm(x, (h0, c0))
        out = self.linear(hn[-1]).flatten()  # get the output of the last hidden layer
        return out

    def training_step(self, batch, batch_idx):
        X, y = batch
        output = self(X)
        loss = self.loss_function(output, y)
        self.log('train_loss', loss, prog_bar=True, on_epoch=True, on_step=False, logger=True)  # Log the training loss
        return loss

    def validation_step(self, batch, batch_idx):
        self.eval()
        X, y = batch
        output = self(X)
        loss = self.loss_function(output, y)
        self.validation_step_outputs.append(output)
        self.log('val_loss', loss, prog_bar=True, on_epoch=True, on_step=False, logger=True)  # Log the validation loss
        return loss

    def predict_step(self, batch, batch_idx):
        self.eval()
        X, _ = batch
        y_star = self(X)
        return y_star

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [ ]:
L.seed_everything(101)
model_lstm = ShallowRegressionLSTM(
    num_features=len(features),
    num_layers=4,
    hidden_units=60,
    learning_rate=5e-4
)

INFO: Seed set to 101
INFO:lightning.fabric.utilities.seed:Seed set to 101


In [ ]:
summary(model_lstm, input_size=(32, 60, 4))

Layer (type:depth-idx)                   Output Shape              Param #
ShallowRegressionLSTM                    [32]                      --
├─LSTM: 1-1                              [32, 60, 60]              103,680
├─Linear: 1-2                            [32, 1]                   61
Total params: 103,741
Trainable params: 103,741
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 199.07
Input size (MB): 0.03
Forward/backward pass size (MB): 0.92
Params size (MB): 0.41
Estimated Total Size (MB): 1.37

## Train

In [ ]:
# Create a ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Monitor validation loss
    mode='min',           # Save the model with the lowest validation loss
    dirpath='./',         # Directory to save the checkpoints
    filename='best_model',  # Filename for the best model checkpoint
    save_top_k=1,
    verbose=True
)

In [ ]:
data_module.setup(stage='fit') # setup train+validate in datamodule

trainer_lstm = L.Trainer(
    callbacks=[checkpoint_callback],
    accelerator="auto",
    devices="auto",
    max_epochs=100,
)
trainer_lstm.fit(model_lstm, datamodule=data_module)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: UserWarning:

Checkpoint directory /content exists and is not empty.

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | lstm          | LSTM    | 103 K  | train
1 | linear        | Linear  | 61     | train
2 | loss_function | MSELoss | 0      | train
--------------------------------------------------
103 K     Trainable params
0         N

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 106: 'val_loss' reached 0.29400 (best 0.29400), saving model to '/content/best_model.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 0, global step 106: 'val_loss' reached 0.29400 (best 0.29400), saving model to '/content/best_model.ckpt' as top 1


...

INFO: Epoch 97, global step 10388: 'val_loss' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 97, global step 10388: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 98, global step 10494: 'val_loss' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 98, global step 10494: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 99, global step 10600: 'val_loss' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 99, global step 10600: 'val_loss' was not in top 1
INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


## Evaluation

In [ ]:
ystar_col = "Model forecast"

train_eval_loader = DataLoader(train_dataset, batch_size=32, shuffle=False) # set shuffle to 'False'

train_pred_lstm = trainer_lstm.predict(model_lstm, train_eval_loader, ckpt_path="best")
val_pred_lstm = trainer_lstm.predict(model_lstm, data_module.val_dataloader(), ckpt_path="best")
test_pred_lstm = trainer_lstm.predict(model_lstm, data_module.predict_dataloader(), ckpt_path="best")

df_train[ystar_col] = torch.cat(train_pred_lstm, 0)
df_val[ystar_col] =  torch.cat(val_pred_lstm, 0)
df_test[ystar_col] = torch.cat(test_pred_lstm, 0)

df_out = pd.concat((df_train, df_val, df_test))[[target, ystar_col]]

for c in df_out.columns:
    df_out[c] = df_out[c] * target_stdev + target_mean


INFO: Restoring states from the checkpoint path at /content/best_model.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at /content/best_model.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at /content/best_model.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at /content/best_model.ckpt


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO: Restoring states from the checkpoint path at /content/best_model.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at /content/best_model.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at /content/best_model.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at /content/best_model.ckpt


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO: Restoring states from the checkpoint path at /content/best_model.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at /content/best_model.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at /content/best_model.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at /content/best_model.ckpt


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
print(df_out)

             Open_lead1  Model forecast
Date                                   
2004-08-19    50.316402       55.738129
2004-08-20    55.168217       56.894867
2004-08-23    55.412300       58.397949
2004-08-24    52.284027       57.842896
2004-08-25    52.279045       57.877899
...                 ...             ...
2019-09-27  1220.969971     1196.561157
2019-09-30  1219.000000     1192.960693
2019-10-01  1196.979980     1188.840088
2019-10-02  1180.000000     1175.114258
2019-10-03  1191.890015     1170.006958

[3808 rows x 2 columns]


In [ ]:
import numpy as np
import math
from sklearn.metrics import mean_squared_error

def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

print( 'MPAE =', MAPE(df_test['Open_lead1'], df_test['Model forecast']) )
print( 'RMSE =', math.sqrt(mean_squared_error(df_test['Open_lead1'], df_test['Model forecast'])) )

MPAE = 2.1253437375548514
RMSE = 0.09549306136606003


In [ ]:
fig = px.line(df_out, labels={'value': "Open", 'created_at': 'Date'})
fig.add_vline(x=val_start, line_width=4, line_dash="dash")
fig.add_vline(x=test_start, line_width=4, line_dash="dash")
# fig.add_annotation(xref="paper", x=0.75, yref="paper", y=0.8, text="Test set start", showarrow=False)
fig.update_layout(
  template=plot_template, legend=dict(orientation='h', y=1.02, title_text="")
)
fig.show()

# GRU

## The model and learning algorithm

In [ ]:
class ShallowRegressionGRU(L.LightningModule):
    def __init__(self, num_features, hidden_units, num_layers, learning_rate=5e-4):
        super().__init__()
        self.num_features = num_features  # this is the number of features
        self.hidden_units = hidden_units
        self.num_layers = num_layers
        self.learning_rate = learning_rate
        self.validation_step_outputs = []

        self.gru = nn.GRU(
            input_size=num_features,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=self.num_layers
        )


        self.linear = nn.Linear(in_features=self.hidden_units, out_features=1)
        self.loss_function = nn.MSELoss()

    def forward(self, x):
        batch_size = x.shape[0]

        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).to(x.device)

        _, hn = self.gru(x, h0)
        out = self.linear(hn[-1]).flatten()  # get the output of the last hidden layer
        return out

    def training_step(self, batch, batch_idx):
        X, y = batch
        output = self(X)
        loss = self.loss_function(output, y)
        self.log('train_loss', loss, prog_bar=True, on_epoch=True, on_step=False, logger=True)  # Log the training loss
        return loss

    def validation_step(self, batch, batch_idx):
        self.eval()
        X, y = batch
        output = self(X)
        loss = self.loss_function(output, y)
        self.validation_step_outputs.append(output)
        self.log('val_loss', loss, prog_bar=True, on_epoch=True, on_step=False, logger=True)  # Log the validation loss
        return loss

    def predict_step(self, batch, batch_idx):
        self.eval()
        X, _ = batch
        y_star = self(X)
        return y_star

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [ ]:
L.seed_everything(101)
model_gru = ShallowRegressionGRU(
    num_features=len(features),
    num_layers=4,
    hidden_units=60,
    learning_rate=5e-4
)

INFO: Seed set to 101
INFO:lightning.fabric.utilities.seed:Seed set to 101


In [ ]:
summary(model_gru, input_size=(32, 60, 4))

Layer (type:depth-idx)                   Output Shape              Param #
ShallowRegressionGRU                     [32]                      --
├─GRU: 1-1                               [32, 60, 60]              77,760
├─Linear: 1-2                            [32, 1]                   61
Total params: 77,821
Trainable params: 77,821
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 149.30
Input size (MB): 0.03
Forward/backward pass size (MB): 0.92
Params size (MB): 0.31
Estimated Total Size (MB): 1.26

## Train

In [ ]:
# Create a ModelCheckpoint callback
checkpoint_callback2 = ModelCheckpoint(
    monitor='val_loss',  # Monitor validation loss
    mode='min',           # Save the model with the lowest validation loss
    dirpath='./',         # Directory to save the checkpoints
    filename='best_gru_model',  # Filename for the best model checkpoint
    verbose=True
)


In [ ]:
trainer_gru = L.Trainer(
    callbacks=[checkpoint_callback2],
    accelerator="auto",
    devices="auto",
    max_epochs=100,
)
data_module.setup(stage='fit')
trainer_gru.fit(model_gru, datamodule=data_module)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: UserWarning:

Checkpoint directory /content exists and is not empty.

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | gru           | GRU     | 77.8 K | train
1 | linear        | Linear  | 61     | train
2 | loss_function | MSELoss | 0      | train
--------------------------------------------------
77.8 K    Trainable params
0         N

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 106: 'val_loss' reached 0.44494 (best 0.44494), saving model to '/content/best_gru_model.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 0, global step 106: 'val_loss' reached 0.44494 (best 0.44494), saving model to '/content/best_gru_model.ckpt' as top 1


...

INFO: Epoch 97, global step 10388: 'val_loss' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 97, global step 10388: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 98, global step 10494: 'val_loss' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 98, global step 10494: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 99, global step 10600: 'val_loss' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 99, global step 10600: 'val_loss' was not in top 1
INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


## Evaluation

In [ ]:
train_eval_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

ystar_col = "Model forecast"

train_pred_gru = trainer_gru.predict(model_gru, train_eval_loader, ckpt_path="best")
val_pred_gru = trainer_gru.predict(model_gru, data_module.val_dataloader(), ckpt_path="best")
test_pred_gru = trainer_gru.predict(model_gru, data_module.predict_dataloader(), ckpt_path="best")

df_train[ystar_col] = torch.cat(train_pred_gru, 0)
df_val[ystar_col] =  torch.cat(val_pred_gru, 0)
df_test[ystar_col] = torch.cat(test_pred_gru, 0)

df_out = pd.concat((df_train, df_val, df_test))[[target, ystar_col]]

for c in df_out.columns:
    df_out[c] = df_out[c] * target_stdev + target_mean

INFO: Restoring states from the checkpoint path at /content/best_gru_model.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at /content/best_gru_model.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at /content/best_gru_model.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at /content/best_gru_model.ckpt


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO: Restoring states from the checkpoint path at /content/best_gru_model.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at /content/best_gru_model.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at /content/best_gru_model.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at /content/best_gru_model.ckpt


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO: Restoring states from the checkpoint path at /content/best_gru_model.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at /content/best_gru_model.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at /content/best_gru_model.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at /content/best_gru_model.ckpt


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
print(df_out)

             Open_lead1  Model forecast
Date                                   
2004-08-19    50.316402       65.806885
2004-08-20    55.168217       63.403625
2004-08-23    55.412300       63.666687
2004-08-24    52.284027       61.196869
2004-08-25    52.279045       60.446381
...                 ...             ...
2019-09-27  1220.969971     1196.618286
2019-09-30  1219.000000     1193.142456
2019-10-01  1196.979980     1187.054565
2019-10-02  1180.000000     1168.836060
2019-10-03  1191.890015     1170.039551

[3808 rows x 2 columns]


In [ ]:
import numpy as np
import math
from sklearn.metrics import mean_squared_error

def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

print( 'MPAE =', MAPE(df_test['Open_lead1'], df_test['Model forecast']) )
print( 'RMSE =', math.sqrt(mean_squared_error(df_val['Open_lead1'], df_val['Model forecast'])) )

MPAE = 1.9250299816237852
RMSE = 0.0775608516373118


In [ ]:
fig = px.line(df_out, labels={'value': "Open", 'created_at': 'Date'})
fig.add_vline(x=val_start, line_width=4, line_dash="dash")
fig.add_vline(x=test_start, line_width=4, line_dash="dash")
# fig.add_annotation(xref="paper", x=0.75, yref="paper", y=0.8, text="Test set start", showarrow=False)
fig.update_layout(
  template=plot_template, legend=dict(orientation='h', y=1.02, title_text="")
)
fig.show()